<a href="https://colab.research.google.com/github/praveenjune17/bert-summarization/blob/master/summarization_scripts/Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install tensorflow-gpu==2.0.0-beta1
#!pip install -q tf-nightly-2.0-preview
!pip install rouge==0.3.2
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import tempfile

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 348.9MB 102kB/s 
     |████████████████████████████████| 501kB 46.3MB/s 
     |████████████████████████████████| 3.1MB 35.1MB/s 


In [0]:
import shutil
import os
import datetime
from collections import defaultdict
%load_ext tensorboard
import tensorflow as tf
tf.random.set_seed(100)
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from rouge import Rouge
!rm -rf ./logs/ 

if not os.path.exists('/content/write_ROGUE/'):
  os.mkdir('/content/write_ROGUE/')

In [3]:
device_name = tf.test.gpu_device_name()

if not tf.test.is_gpu_available():
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
examples, metadata = tfds.load('cnn_dailymail', with_info=True, as_supervised=True)
train_examples = examples['train']
val_examples = examples['validation']
test_examples = examples['test']
pre_filter_count = 287113 # Number of train examples counted already using the below code
rouge_all = Rouge()
# count=0
# for doc, summ in examples['train']:
#   count+=1
#print(count)   # Number of examples available 287113

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

W0906 03:44:58.677903 140315451373440 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_datasets/core/file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset cnn_dailymail downloaded and prepared to /root/tensorflow_datasets/cnn_dailymail/plain_text/0.0.2. Subsequent calls will reuse this data.


In [0]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.load_from_file('/content/vocab_file_summarization')

In [0]:
BATCH_SIZE = 64
doc_length = 512   #(200,75) worked,  , crashed at 230, 75 (275-305)
summ_length = 70
MAX_LENGTH = summ_length
BUFFER_SIZE = pre_filter_count #number of recs after filtering

In [0]:
def encode(lang1, lang2):
  lang1 = [tokenizer_en.vocab_size] + tokenizer_en.encode(
      lang1.numpy()) + [tokenizer_en.vocab_size+1]

  lang2 = [tokenizer_en.vocab_size] + tokenizer_en.encode(
      lang2.numpy()) + [tokenizer_en.vocab_size+1]
  
  return lang1, lang2

# Set threshold for document and  summary length
def filter_max_length(x, y):
  return tf.logical_and(tf.size(x) <= doc_length,
                        tf.size(y) <= summ_length)

def tf_encode(doc, summary):
  return tf.py_function(encode, [doc, summary], [tf.int64, tf.int64])

In [0]:
train_dataset = train_examples.map(tf_encode)
train_dataset = train_dataset.filter(filter_max_length)
#print('Number of records in train set after filter', sum(1 for _, _ in train_dataset))
# 835 recs with doc_length = 180 & summ_length = 75
# 3225 recs with doc_length = 250 & summ_length = 75
# 2250 recs with doc_length = 225 & summ_length = 75
# 1350 recs with doc_length = 240 & summ_length = 50
# 1488 recs with doc_length = 220 & summ_length = 60
# 29K                          512, 70
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE, seed = 100).padded_batch(
    BATCH_SIZE, padded_shapes=([-1], [-1]))
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)


val_dataset = val_examples.map(tf_encode)
val_dataset = val_dataset.filter(filter_max_length).padded_batch(
    BATCH_SIZE, padded_shapes=([-1], [-1]))
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
# doc_batch, sum_batch = next(iter(val_dataset))
# for doc_ids,sum_ids in zip(doc_batch, sum_batch):
#   en_sent = tokenizer_en.decode([i for i in doc_ids if i < tokenizer_en.vocab_size])
#   ta_sent = tokenizer_en.decode([i for i in sum_ids if i < tokenizer_en.vocab_size])
#   print(en_sent, ta_sent)

In [0]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # apply sin to even indices in the array; 2i
  sines = np.sin(angle_rads[:, 0::2])
  
  # apply cos to odd indices in the array; 2i+1
  cosines = np.cos(angle_rads[:, 1::2])
  
  pos_encoding = np.concatenate([sines, cosines], axis=-1)
  
  pos_encoding = pos_encoding[np.newaxis, ...]
    
  return tf.cast(pos_encoding, dtype=tf.float32)

In [0]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions so that we can add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [0]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)  #(1 - lower_triangular_matrix)
  return mask  # (seq_len, seq_len)

In [0]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [0]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

In [0]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [0]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2

In [0]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)
 
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)
    
    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)
    
    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)
    
    return out3, attn_weights_block1, attn_weights_block2

In [0]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(input_vocab_size, self.d_model)
    
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]
    
    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))  #(refer last line of 3.4 Embeddings and Softmax)
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    
    return x  # (batch_size, input_seq_len, d_model)

In [0]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, 
               rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(target_vocab_size, self.d_model)
    
    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}
    
    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]
    
    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)
      
      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [0]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               target_vocab_size, rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                           input_vocab_size, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                           target_vocab_size, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
  def call(self, inp, tar, training, enc_padding_mask, 
           look_ahead_mask, dec_padding_mask):

    enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)
    
    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)
    
    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
    
    return final_output, attention_weights

In [0]:
num_layers = 3  #number of transformer blocks
d_model = 512   #the projected word vector dimension
dff = 512       #
num_heads = 8   #the number of heads in the multi-headed attention unit

input_vocab_size = tokenizer_en.vocab_size + 2
target_vocab_size = input_vocab_size
dropout_rate = 0.0
epsilon_ls = 0.1 #label_smoothing hyper parameter

print('vocab size is {} '.format(input_vocab_size))

In [0]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
    
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [0]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)
loss_object = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction='none')

In [0]:
def label_smoothing(inputs, epsilon=epsilon_ls):
    V = inputs.get_shape().as_list()[-1] # number of channels
    return ((1-epsilon) * inputs) + (epsilon / V)

In [0]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  real = label_smoothing(tf.one_hot(real, depth=tokenizer_en.vocab_size+2))
  #real = (tf.one_hot(real, depth=tokenizer_en.vocab_size+2))
  # pred shape =  (batch_size, tar_seq_len, target_vocab_size)
  # real shape =  (batch_size, tar_seq_len, target_vocab_size)
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_mean(loss_)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')
validation_loss = tf.keras.metrics.Mean(name='validation_loss')
validation_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='validation_accuracy')

In [0]:
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, target_vocab_size, dropout_rate)

In [0]:
def create_masks(inp, tar):
  # Encoder padding mask
  enc_padding_mask = create_padding_mask(inp)  #output 1 if padded 0 is present else 0
  
  # Used in the 2nd attention block in the decoder.
  # This padding mask is used to mask the encoder outputs.
  dec_padding_mask = create_padding_mask(inp)
  # Used in the 1st attention block in the decoder.
  # It is used to pad and mask future tokens in the input received by 
  # the decoder.allows decoder to attend to all positions in the decoder up to and including that position(refer architecture)
  dec_target_padding_mask = create_padding_mask(tar)
  
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
  return enc_padding_mask, combined_mask, dec_padding_mask

In [0]:
checkpoint_path = "/content/drive/My Drive/Neural_machine_translation/Text_summarization_testing"
#checkpoint_path = "/content/checkpoints"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print (ckpt_manager.latest_checkpoint, 'checkpoint restored!!')

In [0]:
EPOCHS = 20

In [0]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]

@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
  
  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp, 
                                 True, 
                                 enc_padding_mask, 
                                 combined_mask, 
                                 dec_padding_mask)
    
    loss = loss_function(tar_real, predictions)
    
    
  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  train_loss(loss)
  train_accuracy(tar_real, predictions)  

In [0]:
def create_temp_file( text):
    temp_file = tempfile.NamedTemporaryFile(delete=False)
    with tf.io.gfile.GFile(temp_file.name, "w") as w:
      w.write(text)
    return temp_file.name

In [0]:
def write_summary(tar_real, predictions, inp, epoch, write=False):
  r_avg_final = []
  if write == False:
    for i, sub_tar_real in enumerate(tar_real):
      predicted_id = tf.cast(tf.argmax(predictions[i], axis=-1), tf.int32)
      sum_ref = tokenizer_en.decode([i for i in sub_tar_real if i < tokenizer_en.vocab_size])
      sum_hyp = tokenizer_en.decode([i for i in predicted_id if i < tokenizer_en.vocab_size])
      rouges = rouge_all.get_scores(sum_ref , sum_hyp)[0]
      r1_val, r2_val, rl_val = rouges['rouge-1']["f"], rouges['rouge-2']["f"], rouges['rouge-l']["f"]
      r_avg = np.mean([r1_val, r2_val, rl_val], dtype=np.float64)
      r_avg_final.append(r_avg)
      score =  np.mean(r_avg_final, dtype=np.float64)
    return score
  else:
    with tf.io.gfile.GFile('/content/write_ROGUE/summary_for_epoch_'+str(epoch.numpy()), 'a') as f:
      for i, sub_tar_real in enumerate(tar_real):
        predicted_id = tf.cast(tf.argmax(predictions[i], axis=-1), tf.int32)
        sum_ref = tokenizer_en.decode([i for i in sub_tar_real if i < tokenizer_en.vocab_size])
        sum_hyp = tokenizer_en.decode([i for i in predicted_id if i < tokenizer_en.vocab_size])
        f.write(sum_ref+'\t'+sum_hyp+'\n')
        rouges = rouge.get_scores(sum_ref , sum_hyp)[0]
        r1_val, r2_val, rl_val = rouges['rouge-1']["f"], rouges['rouge-2']["f"], rouges['rouge-l']["f"]
        r_avg = np.mean([r1_val, r2_val, rl_val], dtype=np.float64)
        r_avg_final.append(r_avg)
      score =  np.mean(r_avg_final, dtype=np.float64)
    return score

def tf_write_summary(tar_real, predictions, inp, epoch):
  return tf.py_function(write_summary, [tar_real, predictions, inp, epoch], Tout=tf.float32)

In [0]:

val_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None), dtype=tf.int32),
]

@tf.function(input_signature=val_step_signature)
def val_step(inp, tar, epoch):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
  
  
  predictions, _ = transformer(inp, tar_inp, 
                               False, 
                               enc_padding_mask, 
                               combined_mask, 
                               dec_padding_mask)
  loss = loss_function(tar_real, predictions)
  
  validation_loss(loss)
  validation_accuracy(tar_real, predictions)
  return tf_write_summary(tar_real, predictions, inp[:, 1:], epoch)
  

def calc_validation_loss(validation_dataset, epoch):
  validation_loss.reset_states()
  validation_accuracy.reset_states()
  val_acc = 0
  val_loss = 0
  
  for (batch, (inp, tar)) in enumerate(validation_dataset):
    rouge = val_step(inp, tar, epoch)
    val_loss += validation_loss.result()
    val_acc += validation_accuracy.result()
  #if val_acc and val_loss:
  return (val_acc.numpy()/(batch+1), val_loss.numpy()/(batch+1), rouge/(batch+1))
  #else:
  #  return (val_acc, val_loss, rouge)

In [0]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
validation_log_dir = 'logs/gradient_tape/' + current_time + '/validation'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
valid_summary_writer = tf.summary.create_file_writer(validation_log_dir)

In [0]:
%tensorboard --logdir logs/gradient_tape

#### Cross check steps for label smoothing

In [0]:
# tar_inp = tar[:, :-1]
# tar_real = tar[:, 1:]
# enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
# predictions, _ = transformer(inp, tar_inp, 
#                              False, 
#                              enc_padding_mask, 
#                              combined_mask, 
#                              dec_padding_mask)
# #one_hot_y_true = tf.one_hot(tar_real, depth=tokenizer_en.vocab_size+2)
# one_hot_y_true = label_smoothing(tf.one_hot(tar_real, depth=tokenizer_en.vocab_size+2))
# cce = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.0, from_logits=True, reduction='none')
# loss_cce = cce(one_hot_y_true, predictions)
# mask = tf.math.logical_not(tf.math.equal(tar_real, 0))
# mask = tf.cast(mask, dtype=loss_cce.dtype)
# loss_cce *= mask
# print('cce Loss: ', tf.reduce_mean(loss_cce).numpy())  
# ####################
# sce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
# loss_sce = sce(tar_real,predictions)
# mask = tf.math.logical_not(tf.math.equal(tar_real, 0))
# mask = tf.cast(mask, dtype=loss_sce.dtype)
# loss_sce *= mask

# print('sce Loss: ', tf.reduce_mean(loss_sce).numpy())  
  

In [0]:
EPOCHS = 160
print_chks = 50

for epoch in range(EPOCHS):
  
  start = time.time()  
  train_loss.reset_states()
  train_accuracy.reset_states()
  validation_loss.reset_states()
  validation_accuracy.reset_states()
  
  # inp -> english, tar -> tamil
  for (batch, (inp, tar)) in enumerate(train_dataset):
    train_step(inp, tar)
    if batch==0 and epoch ==0:
      print('Time taken to feed the input data to the model {} seconds'.format(time.time()-start))
    if batch % print_chks == 0:
      print ('Epoch {} Batch {} Train_Loss {:.4f} Train_Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
  
  (val_acc, val_loss, rouge) = calc_validation_loss(val_dataset, epoch+1)
  ckpt_save_path = ckpt_manager.save()
  
  with train_summary_writer.as_default():
    tf.summary.scalar('train_loss', train_loss.result(), step=epoch)
    tf.summary.scalar('train_accuracy', train_accuracy.result(), step=epoch)
  
  with valid_summary_writer.as_default():
    tf.summary.scalar('validation_loss', validation_loss.result(), step=epoch)
    tf.summary.scalar('validation_accuracy', validation_accuracy.result(), step=epoch)
    tf.summary.scalar('validation_total_loss', val_acc, step=epoch)
    tf.summary.scalar('validation_total_accuracy', val_loss, step=epoch)
    tf.summary.scalar('ROUGE', rouge, step=epoch)
  
  model_metrics = 'Epoch {}, Train Loss: {:.4f}, Train_Accuracy: {:.4f},\
                   Valid Loss: {:.4f},\
                   Valid Accuracy: {:4f}, ROUGE {:.4f}'
  epoch_timing  = 'Time taken for {} epoch : {} secs' 
  validation_run_details = 'Loss {:.4f}, ROUGE {:.4f}, and Accuracy {:.4f}\
                            on the entire validation dataset'
  checkpoint_details = 'Saving checkpoint for epoch {} at {}'
  
  print(model_metrics.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result(),
                         val_loss, 
                         val_acc,
                         rouge))
  print(epoch_timing.format(epoch + 1, time.time() - start))
  print(checkpoint_details.format(epoch+1, ckpt_save_path))

W0905 12:33:59.922390 139929770882816 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0905 12:33:59.924740 139929770882816 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0905 12:33:59.945026 139929762490112 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0905 12:33:59.946947 139929762490112 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0905 12:33:59.986932 139929770882816 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string


Time taken to feed the input data to the model 1386.7471911907196 seconds
Epoch 1 Batch 0 Train_Loss 4.0139 Train_Accuracy 0.2013
Epoch 1 Batch 50 Train_Loss 3.9641 Train_Accuracy 0.2209
Epoch 1 Batch 100 Train_Loss 3.9398 Train_Accuracy 0.2210
Epoch 1 Batch 150 Train_Loss 3.9231 Train_Accuracy 0.2222
Epoch 1 Batch 200 Train_Loss 3.9066 Train_Accuracy 0.2240
Epoch 1 Batch 250 Train_Loss 3.8908 Train_Accuracy 0.2255
Epoch 1 Batch 300 Train_Loss 3.8696 Train_Accuracy 0.2269
Epoch 1 Batch 350 Train_Loss 3.8547 Train_Accuracy 0.2285
Epoch 1 Batch 400 Train_Loss 3.8372 Train_Accuracy 0.2302
Epoch 1 Batch 450 Train_Loss 3.8234 Train_Accuracy 0.2317
Epoch 1, Train Loss: 3.8216, Train_Accuracy: 0.2319,                   Valid Loss: 3.8040,                   Valid Accuracy: 0.240031, ROUGE 0.0111
Time taken for 1 epoch : 2857.180410861969 secs
Saving checkpoint for epoch 1 at /content/drive/My Drive/Neural_machine_translation/Text_summarization_testing/ckpt-5


In [74]:
inp_sentences = ["I'm Batman", "I'm Superman", "I'm The Flash"]
beam_size = 3
N = len(max(inp_sentences, key=len))
new_inp_sentences=[]
new = []
inp_sentences = [tokenizer_en.encode(i) for i in inp_sentences]
for i in inp_sentences:
  if len(i) < N:
    i = [tokenizer_en.vocab_size] + (i + [0] * (N - len(i))) + [tokenizer_en.vocab_size + 1]
  else:
    i = [tokenizer_en.vocab_size] + i + [tokenizer_en.vocab_size + 1]
  new_inp_sentences.append(i)
#inp_sentences = inp_sentences + [0] * (N - len(inp_sentences))
for i in new_inp_sentences:
  new=new + [i]*beam_size
encoder_input = tf.convert_to_tensor(new)
#encoder_input = tf.concat([inp_sentences]*beam_size, axis=0)
print(encoder_input)

tf.Tensor(
[[8261 8078 8044  122 8071  428  223    0    0    0    0    0    0    0
  8262]
 [8261 8078 8044  122 8071  428  223    0    0    0    0    0    0    0
  8262]
 [8261 8078 8044  122 8071  428  223    0    0    0    0    0    0    0
  8262]
 [8261 8078 8044  122 8088  228 7497  214    0    0    0    0    0    0
  8262]
 [8261 8078 8044  122 8088  228 7497  214    0    0    0    0    0    0
  8262]
 [8261 8078 8044  122 8088  228 7497  214    0    0    0    0    0    0
  8262]
 [8261 8078 8044  122 8089   16 8075 7071    0    0    0    0    0    0
  8262]
 [8261 8078 8044  122 8089   16 8075 7071    0    0    0    0    0    0
  8262]
 [8261 8078 8044  122 8089   16 8075 7071    0    0    0    0    0    0
  8262]], shape=(9, 15), dtype=int32)


In [10]:
encoder_input

<tf.Tensor: id=170, shape=(5, 8), dtype=int32, numpy=
array([[8261, 8078, 8044,  122, 8071,  428,  223, 8262],
       [8261, 8078, 8044,  122, 8071,  428,  223, 8262],
       [8261, 8078, 8044,  122, 8071,  428,  223, 8262],
       [8261, 8078, 8044,  122, 8071,  428,  223, 8262],
       [8261, 8078, 8044,  122, 8071,  428,  223, 8262]], dtype=int32)>

In [0]:
#loss = 0.8285# 0.0804 :- validation accuracy
Epoch 13, Loss: 0.5874, Train_Accuracy: 0.1382, Valid Loss: 0.6716, Valid Accuracy: 0.102419, BLEU 0.0589
Time taken for 13 epoch : 344.3424668312073 secs
Saving checkpoint for epoch 13 at /content/checkpoints/train/ckpt-13
# # for (batch, (inp, tar)) in enumerate(sample_train_dataset):
# #   bleu = val_step(inp, tar, epoch)
# #   with valid_summary_writer.as_default():
# #     tf.summary.scalar('BLEU', bleu, step=batch)
# #     tf.summary.scalar('validation_loss', validation_loss.result(), step=batch)
# #     tf.summary.scalar('validation_accuracy', validation_accuracy.result(), step=batch)
      
# (val_acc, val_loss, bleu) = calc_validation_loss(sample_train_dataset, epoch+1)
# ckpt_save_path = ckpt_manager.save()
# model_metrics = 'Epoch {}, Loss: {:.4f}, Accuracy: {:.4f}, Valid Loss: {:.4f}, Valid Accuracy: {:4f}'
# epoch_timing  = 'Time taken for {} epoch : {} secs' 
# validation_run_details = 'Loss {:.4f}, BLEU {:.4f}, and Accuracy {:.4f} on the entire validation dataset'
# checkpoint_details = 'Saving checkpoint for epoch {} at {}'

# print(model_metrics.format(epoch+1,
#                        train_loss.result(), 
#                        train_accuracy.result(),
#                        validation_loss.result(), 
#                        validation_accuracy.result()))
# print(epoch_timing.format(epoch + 1, time.time() - start))
# print(validation_run_details.format(val_loss, bleu, val_acc))
# print(checkpoint_details.format(epoch+1, ckpt_save_path))

In [0]:
#def beam_search_eval(inp_sentence):
#   beam_size = 5
#   start_token = [tokenizer_en.vocab_size]
#   end_token = [tokenizer_en.vocab_size + 1]
#   inp_sentence = start_token + tokenizer_en.encode(inp_sentence) + end_token
#   encoder_input = tf.expand_dims(inp_sentence, 0)
#   encoder_input = tf.concat([encoder_input]*beam_size, axis=0)
#   start = tokenizer_ta.vocab_size
#   end = tokenizer_ta.vocab_size+1
#   def transformer_query(output):

#     enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
#           encoder_input, output)
#     predictions, attention_weights = transformer(encoder_input, 
#                                                  output,
#                                                  False,
#                                                  enc_padding_mask,
#                                                  combined_mask,
#                                                  dec_padding_mask)


#     return (predictions[:,-1:,:])
#   return beam_search(transformer_query, [start], beam_size, 120, target_vocab_size, 0.6, stop_early=False, eos_id=[tokenizer_ta.vocab_size+1])

In [0]:
#kernel_regularizer = tf.keras.regularizers.l2(0.001) L2 regularizer

In [0]:
# highlight the mistakes done.
# wordpiece subword models
# beam search
# Transformer encoder decoder
# data used from TFDS . gathered and accumulated by me .preprocess done in TFDS
# use test set given in the IIT paper

In [0]:
# Post on linkedin to be completed on Aug-12
  
  #In the script to be shared attach a toy dataset to be trained by the users
  #link of my current code needs to be attached  (#optimize the validation step function, #remove my notes)
  #create code to download the weights and vocab path from gdrive
  #ask for parallel corpus data and specify the links I have used for training 
    ## data collection :- tamil & english bilingual books, channels like discovery, history 18 (no fantasy pls)
  #Increase MAXlen to 100 or 200 and train 
  # Why MT ? uses impact. 
  
  #eng_tam_NMT.ipynb :- exhibition
  #Train_eng_tam_NMT.ipynb :- trains the model

In [0]:
#create skeleton (create a loopy version)
                                
  
  #) Set early stop if val loss is not decreasing but train loss is decreasing
  #) Hyper parameter tuning using tensorboard
  #) take out the randomly intilaised weights & biases of the final dense layer and 
      #  multiply it with the target vocab size and then apply sparse cat entropy and check the loss
  
  #) attention visualization (add intensity between two subwords when they are combined to form a word)    (Catcher)
  #) save all the checkpoints(promising) . atleast copy them to harddisk
  #) wrapping the model with tf estimator and train in on TPU   
  #) preprocess tamil stop words https://github.com/AshokR/TamilNLP/blob/master/Resources/TamilStopWords.txt
  
#result :- Best model parameters that suits the data to be trained, confident about the flow, skeleton done

In [0]:
#explore
   
  # word piece encoding                                https://arxiv.org/pdf/1609.08144.pdf                 (reads)
  # beam search best practices                         https://www.aclweb.org/anthology/P18-2054           (reads)                      
  # how BERT works..deconstruct BERT                                                                        (reads)
  # Text augumentation theasurus for word in the sentence                                                   (extra data)
  # using back translation (mini ta to eng model output can be used as extra data)                          (extra data)                 
  # create similar sentences using Word2vec fasttext or glove                                               (extra data)
  # extra data for the model ways to split the exsisting data or try high computational resource like TPU or high end GPU  (extra data)   
  # Watch stanford lectures on Subword models                                                                   (reads)
  # optimize the exsisting architecture reduce parameters (avg spooling in the final layer instead of dense)?   (reads)
  # check where dropout is applied before(causes problems) or after BN                                          (reads)
  # Maintain balance between copying and translating (manning's paper ) ..can be used for copying person's 
                                                                                              #name, place, org ( reads)
# ideas
  # metric:- create en-ta and ta-en model and then compare the output of the embeddings of the en input and processed en output and calculate their cosine similarity